In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"
cohort = "GSE163211"

# Input paths
in_trait_dir = "../../input/GEO/Liver_cirrhosis"
in_cohort_dir = "../../input/GEO/Liver_cirrhosis/GSE163211"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/GSE163211.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/GSE163211.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/GSE163211.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Distinct hepatic gene expression patterns characterize progressive disease in NAFLD"
!Series_summary	"The pathogenesis of non-alcoholic fatty liver disease is not fully understood. Transcriptomic analysis of a large cohort of 318 patients provides evidence of gene perturbations related to inflammation, complement and coagulation pathways, and tissue remodeling in distinct states of NAFLD."
!Series_overall_design	"Using the Nanostring nCounter assay, we quantified expression of 795 genes (and 5 housekeeping genes) hypothesized to be involved in hepatic fibrosis, inflammation, and steatosis in liver tissue from 318 adults. Liver specimens were categorized into 4 distinct NAFLD phenotypes: normal liver histology (NLH), steatosis only (steatosis), non-alcoholic steatohepatitis without fibrosis (NASH F0), and NASH with fibrosis stage 1-4 (NASH F1-F4)."
Sample Characteristics Dictionary:
{0: ['tissue: Human liver'], 1: ['race: Black or African American'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from 
# Nanostring nCounter assay measuring 795 genes in liver tissue.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Liver_cirrhosis), we need to analyze NAFLD stage data
trait_row = 8  # 'nafld stage' corresponds to liver disease status

# For age, it's clearly available
age_row = 3  # 'age' is directly available

# For gender, it's available as 'Sex'
gender_row = 4  # 'Sex' field contains gender information

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert NAFLD stage value to binary indicator for liver cirrhosis."""
    if value is None:
        return None
    
    # Extract the value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # In NAFLD, cirrhosis is the advanced stage of NASH with fibrosis
    # NASH_F1_F4 includes cirrhosis (F4) and advanced fibrosis stages
    if value == "NASH_F1_F4":
        return 1  # Represents presence of cirrhosis or advanced fibrosis
    elif value in ["Normal", "Steatosis", "NASH_F0"]:
        return 0  # Represents absence of cirrhosis or significant fibrosis
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value."""
    if value is None:
        return None
    
    # Extract the value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)  # Convert to numeric value
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    
    # Extract the value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM4974854': [0.0, 35.0, 0.0], 'GSM4974855': [0.0, 33.0, 0.0], 'GSM4974856': [0.0, 55.0, 0.0], 'GSM4974857': [0.0, 49.0, 0.0], 'GSM4974858': [0.0, 27.0, 0.0], 'GSM4974859': [0.0, 44.0, 0.0], 'GSM4974860': [0.0, 58.0, 0.0], 'GSM4974861': [0.0, 53.0, 0.0], 'GSM4974862': [0.0, 46.0, 1.0], 'GSM4974863': [0.0, 61.0, 0.0], 'GSM4974864': [0.0, 29.0, 0.0], 'GSM4974865': [0.0, 19.0, 1.0], 'GSM4974866': [1.0, 33.0, 1.0], 'GSM4974867': [0.0, 64.0, 1.0], 'GSM4974868': [0.0, 16.0, 0.0], 'GSM4974869': [0.0, 25.0, 0.0], 'GSM4974870': [0.0, 72.0, 0.0], 'GSM4974871': [0.0, 24.0, 0.0], 'GSM4974872': [1.0, 60.0, 0.0], 'GSM4974873': [1.0, 58.0, 0.0], 'GSM4974874': [1.0, 70.0, 1.0], 'GSM4974875': [1.0, 33.0, 0.0], 'GSM4974876': [1.0, 44.0, 1.0], 'GSM4974877': [0.0, 29.0, 1.0], 'GSM4974878': [0.0, 46.0, 1.0], 'GSM4974879': [0.0, 58.0, 0.0], 'GSM4974880': [1.0, 16.0, 0.0], 'GSM4974881': [1.0, 32.0, 1.0], 'GSM4974882': [1.0, 17.0, 0.0], 'GSM4974883': [0.0, 44.0, 0.0],

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_cirrhosis/GSE163211/GSE163211_series_matrix.txt.gz
Gene data shape: (800, 318)
First 20 gene/probe identifiers:
Index(['A1BG', 'A2M', 'A2ML1', 'AARS', 'ABCC4', 'ABCG5', 'ABCG8', 'ABLIM1',
       'ABO', 'ACAN', 'ACOT11', 'ACOT2', 'ACSM3', 'ACTA2', 'ACTR2', 'ADAM10',
       'ADAM12', 'ADAM15', 'ADAM17', 'ADAM19'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the gene expression data appear to be human gene symbols already.
# They include well-known gene symbols like A1BG, A2M, ACTA2, etc., which are standard human gene symbols.
# There's no need to map these identifiers to gene symbols as they already are in the correct format.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to liver fibrosis progression, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for liver cirrhosis analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (800, 318)
Gene data shape after normalization: (798, 318)


Normalized gene expression data saved to ../../output/preprocess/Liver_cirrhosis/gene_data/GSE163211.csv
Original clinical data preview:
         !Sample_geo_accession                       GSM4974854  \
0  !Sample_characteristics_ch1              tissue: Human liver   
1  !Sample_characteristics_ch1  race: Black or African American   
2  !Sample_characteristics_ch1                       bmi: 45.77   
3  !Sample_characteristics_ch1                          age: 35   
4  !Sample_characteristics_ch1                      Sex: Female   

            GSM4974855                       GSM4974856  \
0  tissue: Human liver              tissue: Human liver   
1          race: White  race: Black or African American   
2            bmi: 42.8                       bmi: 40.44   
3              age: 33                          age: 55   
4          Sex: Female                      Sex: Female   

                        GSM4974857                       GSM4974858  \
0              tissue: Human liver

Data shape after handling missing values: (318, 801)
For the feature 'Liver_cirrhosis', the least common label is '1.0' with 82 occurrences. This represents 25.79% of the dataset.
The distribution of the feature 'Liver_cirrhosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 35.0
  50% (Median): 44.0
  75%: 53.0
Min: 16.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 75 occurrences. This represents 23.58% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data shape after removing biased features: (318, 801)


Linked data saved to ../../output/preprocess/Liver_cirrhosis/GSE163211.csv
